## Imports

In [ ]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

%matplotlib inline
pd.set_option('display.max_colwidth', 300)

## Load CSV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
games = pd.read_csv("/content/drive/MyDrive/SAC/book_new.csv", sep = ',', header = None)
games.head(8193)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64
0,name,description,2D,3D,Action,Adventure,Arcade,Baseball,Basketball,Beat-'Em-Up,Billiards,Bowling,Boxing,Card Game,Compilation,Cricket,Driving/Racing,Edutainment,Fighting,First-Person,Fitness,Fixed-Screen,Flight,Football (American),Free-to-Play,Gambling,Golf,Hidden Object,Hockey,Hunting/Fishing,Light-Gun,MMO,MOBA,Management,Matching/Stacking,Miscellaneous,Music/Rhythm,On-Rails,Open-World,Party/Minigame,Pinball,Platformer,Puzzle,Real-Time,Roguelike,Role-Playing,Shoot-'Em-Up,Shooter,Simulation,Skateboarding/Skating,Snowboarding/Skiing,Soccer,Sports,Strategy,Survival,Tactical,Team-Based,Tennis,Text-Based,Third-Person,Trivia/Board Game,Turn-Based,VR,Wakeboarding/Surfing,Wrestling
1,Command & Conquer,"Welcome to the New World Order. A gritty high tech world where the art of electronic intelligence and covert surveillance reigns supreme. Where guerrilla strategies and savage combat are the norm. Build bases, muster forces and lacerate your enemies to the bone. All for the love of power. In the...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Return of Arcade,"For nostalgia buffs, these old games are worth your time and money, though you will miss the feel of the arcade controller.",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Dig,"From the combined talents of LucasArts and legendary filmmaker Steven Spielberg comes an epic adventure that plunges headlong into the very core of the unknown. And takes you with it.. In almost every sense, The Dig represents a leap backwards from LucasArts' previous group of adventure games.",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Mission Critical,"Put simply, Mission Critical was one of the best adventures of 1995.",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8188,"10,000 Bullets","The Power of ""10.000 Bullets"".\n\nEnter Crow and Judas, professional hit men who handle all the ""dirty work"" for the Tonio family, a powerful crime syndicate based in Rome, Italy.\n\n- Use the Bullet Time Feature, pay Attention to the Gun gauge!\n- Replenish Gun Gauge with Combo Attacks, go for ...",0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8189,Gundam 3D Operation,Bandai's giant robots clash in this fully 3D third-person fighting game.,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8190,Gundam Zeta,This game is a turn-based take on Gundam's classic robot combat.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
8191,Monster Farm POP,This game lets you brew up your own monsters by taking snapshots on your phone.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Text cleaning

In [ ]:
def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    text = text.lower() 
    
    return text

In [ ]:
games.shape

(73164, 65)

In [ ]:
type(games)

pandas.core.frame.DataFrame

In [ ]:
descriptions = games[1].apply(lambda x: clean_text(x))
descriptions

0                                                                                                                                                                                                                                                                                                        description
1        welcome to the new world order a gritty high tech world where the art of electronic intelligence and covert surveillance reigns supreme where guerrilla strategies and savage combat are the norm build bases muster forces and lacerate your enemies to the bone all for the love of power in the hyper...
2                                                                                                                                                                                           for nostalgia buffs these old games are worth your time and money though you will miss the feel of the arcade controller
3                   from the combined talents of lucasarts and legendary 

In [ ]:
from nltk.corpus import stopwords

In [ ]:
len(games)

73164

In [ ]:
print(games[64])

0        Wrestling
1                0
2                0
3                0
4                0
           ...    
73159            0
73160            0
73161            0
73162            0
73163            0
Name: 64, Length: 73164, dtype: object


In [ ]:
all_genres = []
names = []
descriptions = []
for i in range(1,len(games)):
  current_genre = []
  names.append(games[0][i])
  descriptions.append(games[1][i])
  for j in range(2,64):
    if games[j][i] == '1' or games[j][i] == 1:
      current_genre.append(games[j][0])
  all_genres.append(current_genre)


print(all_genres)


[['Real-Time', 'Strategy'], ['Compilation'], ['Adventure'], ['Adventure'], ['Golf', 'Simulation', 'Sports'], ['3D', 'Action', 'On-Rails', 'Shooter'], ['Adventure'], ['Arcade', 'Driving/Racing'], ['2D', 'Action', "Beat-'Em-Up"], ['3D', 'Action', 'First-Person', 'Shooter'], ['Flight', 'Simulation'], ['2D', 'Action', 'Platformer'], ['Real-Time', 'Strategy'], ['3D', 'Action', 'First-Person', 'Shooter'], ['Action', 'Role-Playing'], ['3D', 'Action', 'First-Person', 'Shooter'], ['Trivia/Board Game'], ['Trivia/Board Game'], ['Trivia/Board Game'], ['Real-Time', 'Strategy'], ['Strategy', 'Turn-Based'], ['Action', 'Adventure'], ['Adventure'], ['Simulation'], ['Driving/Racing', 'Simulation'], ['Action', 'Role-Playing'], ['Adventure'], ['Flight', 'Simulation'], ['Trivia/Board Game'], ['Real-Time', 'Strategy'], ['Action'], ['Adventure'], ['3D', 'Action', 'First-Person', 'Shooter'], ['Strategy'], ['Strategy', 'Turn-Based'], ['Flight', 'Simulation'], ['3D', 'Action', 'First-Person', 'Shooter'], ['3D',

In [ ]:
print(len(names))
print(len(descriptions))
print(len(all_genres))

73163
73163
73163


In [ ]:
descriptions

["Welcome to the New World Order. A gritty high tech world where the art of electronic intelligence and covert surveillance reigns supreme. Where guerrilla strategies and savage combat are the norm. Build bases, muster forces and lacerate your enemies to the bone. All for the love of power. In the hyper-real combat experience of Command & Conquer, you're thrust into the heat of an all out race for global control. Will you side with the humanistic GDI or bring the world to its knees with the Brotherhood of Nod? Decide fast. Your adversaries are without mercy. And so are you.. Overall Command & Conquer is a great strategy game and easily the best available on the PlayStation.",
 'For nostalgia buffs, these old games are worth your time and money, though you will miss the feel of the arcade controller.',
 "From the combined talents of LucasArts and legendary filmmaker Steven Spielberg comes an epic adventure that plunges headlong into the very core of the unknown.  And takes you with it..

In [ ]:
games_df = pd.DataFrame({'Name': names, 'Description': descriptions, 'Genres': all_genres})
print(games_df)

                                             Name  ...                      Genres
0                               Command & Conquer  ...       [Real-Time, Strategy]
1                                Return of Arcade  ...               [Compilation]
2                                         The Dig  ...                 [Adventure]
3                                Mission Critical  ...                 [Adventure]
4        Links LS: Legends in Sports 1997 Edition  ...  [Golf, Simulation, Sports]
...                                           ...  ...                         ...
73158  The Mermaid's Secret Remedy The Love Spell  ...                 [Adventure]
73159                               Up in the Air  ...         [Action, Adventure]
73160                     Peg Solitaire Adventure  ...         [Trivia/Board Game]
73161                                      Polary  ...            [Action, Arcade]
73162                            Valkyrie's Arrow  ...       [Real-Time, Strategy]

[73

In [ ]:
games_df['Description'] = games_df['Description'].apply(lambda x: clean_text(x))

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
games_df['Description']

0        welcome to the new world order a gritty high tech world where the art of electronic intelligence and covert surveillance reigns supreme where guerrilla strategies and savage combat are the norm build bases muster forces and lacerate your enemies to the bone all for the love of power in the hyper...
1                                                                                                                                                                                           for nostalgia buffs these old games are worth your time and money though you will miss the feel of the arcade controller
2                   from the combined talents of lucasarts and legendary filmmaker steven spielberg comes an epic adventure that plunges headlong into the very core of the unknown and takes you with it in almost every sense the dig represents a leap backwards from lucasarts previous group of adventure games
3                                                                        

In [ ]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# function to remove stopwords
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

games_df['Description'] = games_df['Description'].apply(lambda x: remove_stopwords(x))

In [ ]:
games_df['Description']

0        welcome new world order gritty high tech world art electronic intelligence covert surveillance reigns supreme guerrilla strategies savage combat norm build bases muster forces lacerate enemies bone love power hyper real combat experience command conquer youre thrust heat race global control side...
1                                                                                                                                                                                                                                      nostalgia buffs old games worth time money though miss feel arcade controller
2                                                                                                 combined talents lucasarts legendary filmmaker steven spielberg comes epic adventure plunges headlong core unknown takes almost every sense dig represents leap backwards lucasarts previous group adventure games
3                                                                        

## Training

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(games_df['Genres'])

# transform target variable
y = multilabel_binarizer.transform(games_df['Genres'])

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(games_df['Description'], y, test_size=0.2, random_state=9)

In [ ]:
print(xtrain)

72400    soundxworld new exciting way experience new exclusive music art live music events virtual world game new musical entertainment experience every update new songs albums new art live performances seasonal events added future soundxworld features different environments view consume virtual music re...
71142                                                                                                                                                               fish simple match game tap screen drag drop fish get high score made construct enjoy game game nice graphics music collect many jelly get points
41490    severed follows saga distraught warrior lost fantastical world knits together pieces story past future heroine players embark journey surreal nonlinear world using gesture based combat mechanic defeat enemies first person adventure journey players unlock new abilities discover secrets land grow ...
54907                   thing worse losing life love hunt philophobia gam

In [ ]:
xtrain_tfidf = tfidf_vectorizer.fit_transform(xtrain)
xval_tfidf = tfidf_vectorizer.transform(xval)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score

In [ ]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [ ]:
# fit model on train data
clf.fit(xtrain_tfidf, ytrain)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


OneVsRestClassifier(estimator=LogisticRegression())

In [ ]:
# make predictions for validation set
y_pred = clf.predict(xval_tfidf)

In [ ]:
y_pred[3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
multilabel_binarizer.inverse_transform(y_pred)[4]

('3D', 'Action', 'Adventure', 'Survival')

In [ ]:
# evaluate performance
f1_score(yval, y_pred, average="micro")

0.560080811791175

In [ ]:
# predict probabilities
y_pred_prob = clf.predict_proba(xval_tfidf)

In [ ]:
t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

In [ ]:
# evaluate performance
f1_score(yval, y_pred_new, average="micro")

0.6356724040098355

In [ ]:
def infer_tags(q):
    q = clean_text(q)
    q = remove_stopwords(q)
    q_vec = tfidf_vectorizer.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)

In [ ]:
for i in range(15): 
  k = xval.sample(1).index[0] 
  print("Game: ", games_df['Name'][k], "\nDescription after cleanup: ", games_df['Description'][k], "\nPredicted genre: ", infer_tags(xval[k])), print("Actual genre: ",games_df['Genres'][k], "\n")

In [ ]:
import pickle

filename="/content/drive/MyDrive/SAC/sac_trained_model"
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
filename_decoder="/content/drive/MyDrive/SAC/multilabel_decoder"
pickle.dump(multilabel_binarizer, open(filename_decoder, 'wb'))
vectorizer_file = "/content/drive/MyDrive/SAC/tfidf_vectorizer"
pickle.dump(tfidf_vectorizer, open(vectorizer_file, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))
decoder = pickle.load(open(filename_decoder, 'rb'))
vectorizer = pickle.load(open(vectorizer_file, 'rb'))
y_pred_prob = loaded_model.predict_proba(xval_tfidf)
y_pred_new = (y_pred_prob >= t).astype(int)
f1_score(yval, y_pred_new, average="micro")

0.6356724040098355

In [ ]:
 q = "A game that makes you attack and kill zombies in differente zones with multiple weapons"
q_vec = vectorizer.transform([q])
q_pred = loaded_model.predict(q_vec)
decoder.inverse_transform(q_pred)

[('3D', 'Action', 'Shooter')]